In [14]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os
import pprint
import shutil

import torch
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter

# import _init_paths
import sys
sys.path.append('../monkey_hrnet/hrnet/lib/')

from config import cfg
from config import update_config
from core.loss import JointsMSELoss
from utils.utils import get_optimizer
from utils.utils import save_checkpoint
from utils.utils import create_logger
from utils.utils import get_model_summary

from function import train
from function import validate

# import dataset
import models
# torch.set_default_dtype(torch.float16)
parser = argparse.ArgumentParser(description='Train keypoints network')
# general
parser.add_argument('--cfg',
                    help='experiment configure file name',
                    required=True,
                    type=str)

parser.add_argument('opts',
                    help="Modify config options using the command-line",
                    default=None,
                    nargs=argparse.REMAINDER)

# philly
parser.add_argument('--modelDir',
                    help='model directory',
                    type=str,
                    default='')
parser.add_argument('--logDir',
                    help='log directory',
                    type=str,
                    default='')
parser.add_argument('--dataDir',
                    help='data directory',
                    type=str,
                    default='')
parser.add_argument('--prevModelDir',
                    help='prev Model directory',
                    type=str,
                    default='')

#args = parser.parse_args()
args = parser.parse_args(['--cfg', '/home/myp7435/monkey_hrnet/hrnet/experiments/coco/hrnet/w32_384x288_adam_lr1e-3.yaml'])
#                          '--modelDir', '/home/myp7435/monkey_hrnet/hrnet/output/coco/pose_hrnet/w32_384x288_adam_lr1e-3/model_best.pth'])
# args = parser.parse_args(['--modelDir', '/home/myp7435/monkey_hrnet/hrnet/output/coco/pose_hrnet/w32_384x288_adam_lr1e-3/model_best.pth'])

update_config(cfg, args)

logger, final_output_dir, tb_log_dir = create_logger(
    cfg, args.cfg, 'train')

logger.info(pprint.pformat(args))
logger.info(cfg)

# cudnn related setting
cudnn.benchmark = cfg.CUDNN.BENCHMARK
torch.backends.cudnn.deterministic = cfg.CUDNN.DETERMINISTIC
torch.backends.cudnn.enabled = cfg.CUDNN.ENABLED


# model = eval('models.'+cfg.MODEL.NAME+'.get_pose_net')(cfg, is_train=True).half()
model = eval('models.'+cfg.MODEL.NAME+'.get_pose_net')(cfg, is_train=False)
model_state_file = os.path.join(
            final_output_dir, 'model_best.pth'
        )
logger.info('=> loading model from {}'.format(model_state_file))
model.load_state_dict(torch.load(model_state_file))

model = torch.nn.DataParallel(model, device_ids=cfg.GPUS).cuda()

criterion = JointsMSELoss(
    use_target_weight=cfg.LOSS.USE_TARGET_WEIGHT
).cuda()

# Data loading code
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
)



Namespace(cfg='/home/myp7435/monkey_hrnet/hrnet/experiments/coco/hrnet/w32_384x288_adam_lr1e-3.yaml', dataDir='', logDir='', modelDir='', opts=[], prevModelDir='')
AUTO_RESUME: False
CUDNN:
  BENCHMARK: True
  DETERMINISTIC: False
  ENABLED: True
DATASET:
  COLOR_RGB: True
  DATASET: coco
  DATA_FORMAT: jpg
  FLIP: False
  HYBRID_JOINTS_TYPE: 
  NUM_JOINTS_HALF_BODY: 8
  PROB_HALF_BODY: 0.3
  ROOT: /home/myp7435/monkey_hrnet/hrnet/data/coco/
  ROT_FACTOR: 45
  SCALE_FACTOR: 0.35
  SELECT_DATA: False
  TEST_SET: val2017
  TRAIN_SET: train2017
DATA_DIR: 
DEBUG:
  DEBUG: True
  SAVE_BATCH_IMAGES_GT: True
  SAVE_BATCH_IMAGES_PRED: True
  SAVE_HEATMAPS_GT: True
  SAVE_HEATMAPS_PRED: True
GPUS: (0, 1)
LOG_DIR: /home/myp7435/monkey_hrnet/hrnet/log/
LOSS:
  TOPK: 8
  USE_DIFFERENT_JOINTS_WEIGHT: False
  USE_OHKM: False
  USE_TARGET_WEIGHT: True
MODEL:
  EXTRA:
    FINAL_CONV_KERNEL: 1
    PRETRAINED_LAYERS: ['conv1', 'bn1', 'conv2', 'bn2', 'layer1', 'transition1', 'stage2', 'transition2', 'sta

=> creating /home/myp7435/monkey_hrnet/hrnet/output/coco/pose_hrnet/w32_384x288_adam_lr1e-3
=> creating /home/myp7435/monkey_hrnet/hrnet/log/coco/pose_hrnet/w32_384x288_adam_lr1e-3_2021-06-20-02-59


=> loading model from /home/myp7435/monkey_hrnet/hrnet/output/coco/pose_hrnet/w32_384x288_adam_lr1e-3/model_best.pth


In [3]:
# copy model file
try:
#     this_dir = os.path.dirname(__file__)
    this_dir = os.getcwd()
except NameError:  # We are the main py2exe script, not a module
    import sys
    this_dir = os.path.abspath(sys.argv[0])

shutil.copy2(
    os.path.join(this_dir, 'hrnet/lib/models', cfg.MODEL.NAME + '.py'),
    final_output_dir)
# logger.info(pprint.pformat(model))

writer_dict = {
    'writer': SummaryWriter(log_dir=tb_log_dir),
    'train_global_steps': 0,
    'valid_global_steps': 0,
}


best_perf = 0.0
best_model = False
last_epoch = -1
optimizer = get_optimizer(cfg, model)
begin_epoch = cfg.TRAIN.BEGIN_EPOCH
checkpoint_file = os.path.join(
    final_output_dir, 'checkpoint.pth'
)

if cfg.AUTO_RESUME and os.path.exists(checkpoint_file):
    logger.info("=> loading checkpoint '{}'".format(checkpoint_file))
    checkpoint = torch.load(checkpoint_file)
    begin_epoch = checkpoint['epoch']
    best_perf = checkpoint['perf']
    last_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])

    optimizer.load_state_dict(checkpoint['optimizer'])
    logger.info("=> loaded checkpoint '{}' (epoch {})".format(
        checkpoint_file, checkpoint['epoch']))

lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(
    optimizer, cfg.TRAIN.LR_STEP, cfg.TRAIN.LR_FACTOR,
    last_epoch=last_epoch
)

In [19]:
from my_utils import read_dlc_labeled_data
root_dir = '/home/myp7435/downsampled_data'
folders = ['cam_0', 'cam_1', 'cam_2', 'cam_3', 
                 'cam_0_0610', 'cam_1_0610', 'cam_2_0610', 'cam_3_0610', 
                 'cam_0_0811', 'cam_1_0811', 'cam_2_0811', 'cam_3_0811',
                 'cam_0_0811_b5', 'cam_1_0811_b5', 'cam_2_0811_b5', 'cam_3_0811_b5',
                 'cam_0_0811_b6', 'cam_1_0811_b6', 'cam_2_0811_b6', 'cam_3_0811_b6',
                 'cam_0_0827_b1', 'cam_1_0827_b1', 'cam_2_0827_b1', 'cam_3_0827_b1', 
                 'cam_0_0827_b2', 'cam_1_0827_b2', 'cam_2_0827_b2', 'cam_3_0827_b2', 
                 'cam_0_1103_b1', 'cam_1_1103_b1', 'cam_2_1103_b1', 'cam_3_1103_b1', 
                 'cam_0_1103_b2', 'cam_1_1103_b2', 'cam_2_1103_b2', 'cam_3_1103_b2', 
                ]
df = read_dlc_labeled_data(root_dir, folders)
#%%
from CustomDataset import CustomDataset

import numpy as np
idx = np.random.choice(len(df), int(len(df)*0.9), replace=False)
df_train = [df[i] for i in idx]
df_valid = [df[i] for i in np.arange(len(df)) if i not in idx]

train_dataset = CustomDataset(cfg, df_train, 
                              transform=transforms.Compose([transforms.ToTensor(),normalize,]))

valid_dataset = CustomDataset(cfg, df_valid, 
                              transform=transforms.Compose([transforms.ToTensor(),normalize,]))

#%%
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=cfg.TRAIN.BATCH_SIZE_PER_GPU*len(cfg.GPUS),
    shuffle=cfg.TRAIN.SHUFFLE,
    num_workers=cfg.WORKERS,
    pin_memory=cfg.PIN_MEMORY
)

valid_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=cfg.TEST.BATCH_SIZE_PER_GPU*len(cfg.GPUS),
    shuffle=False,
    num_workers=cfg.WORKERS,
    pin_memory=cfg.PIN_MEMORY
)

Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.


In [20]:
train_dataset[0]

(tensor([[[-1.6042, -1.6042, -1.5870,  ..., -1.4843, -1.4843, -1.5014],
          [-1.5699, -1.5699, -1.5870,  ..., -1.4843, -1.4843, -1.5014],
          [-1.5185, -1.5185, -1.5185,  ..., -1.4843, -1.4843, -1.5014],
          ...,
          [-1.4158, -1.4158, -1.3987,  ..., -1.5357, -1.5357, -1.5357],
          [-1.4158, -1.4158, -1.3987,  ..., -1.5357, -1.5357, -1.5357],
          [-1.4158, -1.4158, -1.3987,  ..., -1.5357, -1.5357, -1.5357]],
 
         [[-1.4755, -1.4755, -1.4580,  ..., -1.3529, -1.3529, -1.3704],
          [-1.4405, -1.4405, -1.4580,  ..., -1.3529, -1.3529, -1.3704],
          [-1.3880, -1.3880, -1.3880,  ..., -1.3529, -1.3529, -1.3704],
          ...,
          [-1.2829, -1.2829, -1.2654,  ..., -1.4055, -1.4055, -1.4055],
          [-1.2829, -1.2829, -1.2654,  ..., -1.4055, -1.4055, -1.4055],
          [-1.2829, -1.2829, -1.2654,  ..., -1.4055, -1.4055, -1.4055]],
 
         [[-1.2467, -1.2467, -1.2293,  ..., -1.1247, -1.1247, -1.1421],
          [-1.2119, -1.2119,

In [ ]:
for epoch in range(begin_epoch, cfg.TRAIN.END_EPOCH):
    lr_scheduler.step()

    # train for one epoch
    train(cfg, train_loader, model, criterion, optimizer, epoch,
          final_output_dir, tb_log_dir, writer_dict)
    best_model = True

#     evaluate on validation set
    perf_indicator = validate(
       cfg, valid_loader, valid_dataset, model, criterion,
       final_output_dir, tb_log_dir, writer_dict
   )

    if perf_indicator >= best_perf:
        best_perf = perf_indicator
        best_model = True
    else:
        best_model = False
    if perf_indicator <= best_perf:
        best_perf = perf_indicator
        best_model = True
    else:
        best_model = False

    logger.info('=> saving checkpoint to {}'.format(final_output_dir))
    save_checkpoint({
        'epoch': epoch + 1,
        'model': cfg.MODEL.NAME,
        'state_dict': model.state_dict(),
        'best_state_dict': model.module.state_dict(),
        'perf': perf_indicator,
        'optimizer': optimizer.state_dict(),
    }, best_model, final_output_dir)

final_model_state_file = os.path.join(
    final_output_dir, 'final_state.pth'
)
logger.info('=> saving final model state to {}'.format(
    final_model_state_file)
)
torch.save(model.module.state_dict(), final_model_state_file)
writer_dict['writer'].close()


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
Epoch: [0][0/7532]	Time 1.660s (1.660s)	Speed 1.2 samples/s	Data 0.100s (0.100s)	Loss 0.00007 (0.00007)	Accuracy 10000.000 (0.000)
Epoch: [0][100/7532]	Time 0.678s (0.702s)	Speed 2.9 samples/s	Data 0.069s (0.072s)	Loss 0.00007 (0.00007)	Accuracy 10000.000 (0.000)
Epoch: [0][200/7532]	Time 0.662s (0.702s)	Speed 3.0 samples/s	Data 0.056s (0.073s)	Loss 0.00007 (0.00007)	Accuracy 10000.000 (0.000)
Epoch: [0][300/7532]	Time 0.708s (0.702s)	Speed 2.8 s

In [12]:
import os
from my_utils import generate_downsampled_video

date = '1217'
save_directory = os.path.join('/home/myp7435/downsampled_videos', date)
if not os.path.exists(save_directory):
    os.mkdir(save_directory)
    exp_save_dir = os.path.join(save_directory, 'experiment')
    calib_save_dir = os.path.join(save_directory, 'calib')
    os.mkdir(exp_save_dir)
    os.mkdir(calib_save_dir)

videos1 = ['cam_0_trimmed.mp4', 'cam_1_trimmed.mp4', 'cam_2_trimmed.mp4', 'cam_3_trimmed.mp4']
videos2 = ['cam_0.avi', 'cam_1.avi', 'cam_2.avi', 'cam_3.avi']

video_directory = os.path.join('/home/myp7435/Pop_freeReach_0317_merged-Min-2020-04-19/videos', date)
calib_directory = '/home/myp7435/Pop_freeReach_0317_merged-Min-2020-04-19/videos/calib_' + date

for vid1, vid2 in zip(videos1, videos2):
    video_path = os.path.join(video_directory, vid1)
    calib_path = os.path.join(calib_directory, vid2)
    exp_save_path = os.path.join(exp_save_dir, vid1)
    calib_save_path = os.path.join(calib_save_dir, vid2.split('.')[0] + '.mp4')
    width = 512
    generate_downsampled_video(video_path, exp_save_path, width)
    generate_downsampled_video(calib_path, calib_save_path, width)

t:   0%|          | 7/3300 [00:00<00:53, 61.85it/s, now=None]

Moviepy - Building video /home/myp7435/downsampled_videos/1217/experiment/cam_0_trimmed.mp4.
Moviepy - Writing video /home/myp7435/downsampled_videos/1217/experiment/cam_0_trimmed.mp4



Moviepy - Done !
Moviepy - video ready /home/myp7435/downsampled_videos/1217/experiment/cam_0_trimmed.mp4
(512.0, 384.0)


t:   0%|          | 6/1800 [00:00<00:33, 53.38it/s, now=None]

Moviepy - Building video /home/myp7435/downsampled_videos/1217/calib/cam_0.mp4.
Moviepy - Writing video /home/myp7435/downsampled_videos/1217/calib/cam_0.mp4



...

Moviepy - Building video /home/myp7435/downsampled_videos/1217/experiment/cam_3_trimmed.mp4.
Moviepy - Writing video /home/myp7435/downsampled_videos/1217/experiment/cam_3_trimmed.mp4



Moviepy - Done !
Moviepy - video ready /home/myp7435/downsampled_videos/1217/experiment/cam_3_trimmed.mp4
(512.0, 384.0)


t:   0%|          | 8/1800 [00:00<00:24, 72.05it/s, now=None]

Moviepy - Building video /home/myp7435/downsampled_videos/1217/calib/cam_3.mp4.
Moviepy - Writing video /home/myp7435/downsampled_videos/1217/calib/cam_3.mp4



Moviepy - Done !
Moviepy - video ready /home/myp7435/downsampled_videos/1217/calib/cam_3.mp4
(512.0, 384.0)


In [17]:
from my_utils import infer_video

joints_name = ['Wrist', 'CMC_thumb', 'MCP_thumb', 'MCP1', 'MCP2', 'MCP3', 'MCP4',
          'IP_thumb', 'PIP1', 'PIP2', 'PIP3', 'PIP4', 'Dip1', 'Dip2', 'Dip3', 'Dip4',
          'Tip_thumb', 'Tip1', 'Tip2', 'Tip3', 'Tip4']
for vid in videos1:
    video_path = os.path.join(exp_save_dir, vid)
    infer_video(video_path, model, joints_name, 
                transform=transforms.Compose([transforms.ToTensor(),normalize,]), downsample=1)

100%|██████████| 3300/3300 [05:26<00:00, 10.10it/s]


In [1]:
from my_utils import create_labeled_video

joints_name = ['Wrist', 'CMC_thumb', 'MCP_thumb', 'MCP1', 'MCP2', 'MCP3', 'MCP4',
          'IP_thumb', 'PIP1', 'PIP2', 'PIP3', 'PIP4', 'Dip1', 'Dip2', 'Dip3', 'Dip4',
          'Tip_thumb', 'Tip1', 'Tip2', 'Tip3', 'Tip4']

for vid in videos1:
    video_path = os.path.join(exp_save_dir, vid)
    infer_video(video_path, model, joints_name, 
                transform=transforms.Compose([transforms.ToTensor(),normalize,]), downsample=1)
create_labeled_video(video_path, csvpath, joints_name)

100%|██████████| 18000/18000 [01:17<00:00, 233.48it/s]
